In [1]:
# Imports
%matplotlib inline
import matplotlib.pyplot as plt
import random
import numpy as np
from time        import process_time 
from itertools   import permutations, combinations
from functools   import lru_cache as cache
from collections import Counter
from statistics  import mean, median

In [2]:
#Definisikan data Cities (random) bagian ini yang harus kita ganti menjadi data pelanggan dari PDAM

alltours = permutations 

def tour_length(tour):
    """The total of distances between each pair of consecutive cities in the tour.
    This includes the last-to-first, distance(tour[-1], tour[0])"""
    return sum(distance(tour[i - 1], tour[i]) 
               for i in range(len(tour)))

City = complex

def distance(A, B): return abs(A - B)

def Cities(n, seed=123, width=999, height=666):
    "Make a set of n cities, sampled uniformly from a (width x height) rectangle."
    random.seed((n, seed))
    return frozenset(City(random.randint(1, width), random.randint(1, height))
                     for c in range(n))

In [3]:
from math import radians, cos, sin, asin, sqrt
#Haversine Formula (untuk mengukur jarak antara 2 kordinat dengan menarik garis lurus pada titik bumi)
def haversine(lon1, lat1, lon2, lat2):
    """
    Calculate the great circle distance in kilometers between two points 
    on the earth (specified in decimal degrees)
    """
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles. Determines return value units.
    return c * r * 100 # Ubah ke meter

In [34]:
#Definisikan Titik Kunjungan
customers = [
             ["1", "Kantor PDAM", -7.548240, 111.658970],
             ["1000000235", "ABDULLAH HASAN, H", -7.6243582, 111.4898561],
             ["1000000322", "SUKRI", -7.6264483, 111.4921134],
             ["1000000228", "SUYANTO", -7.6284169, 111.4949986],
             ["1000000222", "MOCH YASIN", -7.6251153, 111.4870914],
             ["1000000211", "DJUMARI", -7.6246128, 111.4863572]
]

#Bikin Array Jarak
distancesArray = []
for customer in customers:
  collumns = []
  for neighbor in customers :
    #print(customer[1]+" - "+neighbor[1])
    collumns.append(haversine(customer[2], customer[3], neighbor[2], neighbor[3]))
  distancesArray.append(collumns)

for row in distancesArray:
    print(*row)
#haversine(customers[0][2], customers[0][3], customers[1][2], customers[1][2])

0.0 1906.0414195564383 1882.7180520817415 1852.521536262969 1936.8742244217244 1944.598643121538
1906.0414195564383 0.0 26.50483957628938 59.52464246089284 30.896349368680287 38.9198112318175
1882.7180520817415 26.50483957628938 0.0 33.06929615114777 56.10546211806835 64.44122516938322
1852.521536262969 59.52464246089284 33.06929615114777 0.0 88.94680059589619 97.32955031438047
1936.8742244217244 30.896349368680287 56.10546211806835 88.94680059589619 0.0 8.41656122092674
1944.598643121538 38.9198112318175 64.44122516938322 97.32955031438047 8.41656122092674 0.0


In [5]:
#Definisikan fungsi untuk Plot Tampilan, fungsi ini tidak kita pakai lagi kalau sudah menggunakan data PDAM, kita akan menggunakan tampilan dari Google Maps

def plot_tour(tour, style='bo-'): 
    "Plot every city and link in the tour, and highlight start city."
    if len(tour) > 1000: plt.figure(figsize=(15, 10))
    start = tour[0:1]
    plot_segment(tour + start, style)
    plot_segment(start, 'rD') # start city is red Diamond.

def plot_segment(segment, style='bo-'):
    "Plot every city and link in the segment."
    plt.plot([X(c) for c in segment], [Y(c) for c in segment], style, clip_on=False)
    plt.axis('scaled')
    plt.axis('off')

def X(city): "X coordinate."; return city.real
def Y(city): "Y coordinate."; return city.imag

In [6]:
#Function do (bawaan dari contoh kodingan, ini melakukan perhitungan waktu proses, panggil nama algorimanya dan plot tampilan)

def do(algorithm, cities):
    "Apply a TSP algorithm to cities, plot the result, and print info."
    t0   = process_time()
    tour = nn_tsp(cities)
    t1   = process_time()
    assert Counter(tour) == Counter(cities) # Every city appears exactly once in tour
    plot_tour(tour)
    print(tour)
    print("NN: {} cities ⇒ tour length {:.0f} (in {:.3f} sec)".format(len(tour), tour_length(tour), t1 - t0))
    
#def name(algorithm): return algorithm.__name__.replace('_tsp', '')

In [11]:
#Function Nearest Neighbor, ini inti dari Tubes kita

def nn_tsp(cities, start=None):
    """Start the tour at the given start city (default: first city); 
    at each step extend the tour by moving from the previous city 
    to its nearest neighbor that has not yet been visited."""
    C = start or first(cities)
    tour = [C]
    unvisited = set(cities - {C})
    print(unvisited)
    while unvisited:
        C = nearest_neighbor(C, unvisited)
        tour.append(C)
        unvisited.remove(C)
    return tour

def first(collection): return next(iter(collection))

def nearest_neighbor(A, cities):
    "Find the city in cities that is nearest to city A."
    return min(cities, key=lambda C: distance(C, A))

In [71]:
#Panggil Functionnya disini
#Cities(10)
#do(nn_tsp, Cities(10))

tour = []
visited = []
for customer in customers:
  shortestDistance = 0
  for neighbor in customers :
    distance = haversine(customer[2], customer[3], neighbor[2], neighbor[3])
    if(customer[0] != neighbor[0]):
      if customer[0] not in visited:
        if(shortestDistance == 0):
          shortestDistance = distance
          i = customer[1]
          j = neighbor[1]
          print("{} - {} : {:.3f} Meter".format(i, j, distance))
        elif(shortestDistance > distance):
          shortestDistance = distance
          i = customer[1]
          j = neighbor[1]
          print("{} - {} : {:.3f} Meter".format(i, j, distance))
  print("Shortest {} - {} : {:.3f} Meter\n".format(i, j, shortestDistance))
  tour.append([i,j])

#print("\n{} - {} : {:.3f} Meter".format(i, j, distance))
print(tour)


Kantor PDAM - ABDULLAH HASAN, H : 1906.041 Meter
Kantor PDAM - SUKRI : 1882.718 Meter
Kantor PDAM - SUYANTO : 1852.522 Meter
Shortest Kantor PDAM - SUYANTO : 1852.522 Meter

ABDULLAH HASAN, H - Kantor PDAM : 1906.041 Meter
ABDULLAH HASAN, H - SUKRI : 26.505 Meter
Shortest ABDULLAH HASAN, H - SUKRI : 26.505 Meter

SUKRI - Kantor PDAM : 1882.718 Meter
SUKRI - ABDULLAH HASAN, H : 26.505 Meter
Shortest SUKRI - ABDULLAH HASAN, H : 26.505 Meter

SUYANTO - Kantor PDAM : 1852.522 Meter
SUYANTO - ABDULLAH HASAN, H : 59.525 Meter
SUYANTO - SUKRI : 33.069 Meter
Shortest SUYANTO - SUKRI : 33.069 Meter

MOCH YASIN - Kantor PDAM : 1936.874 Meter
MOCH YASIN - ABDULLAH HASAN, H : 30.896 Meter
MOCH YASIN - DJUMARI : 8.417 Meter
Shortest MOCH YASIN - DJUMARI : 8.417 Meter

DJUMARI - Kantor PDAM : 1944.599 Meter
DJUMARI - ABDULLAH HASAN, H : 38.920 Meter
DJUMARI - MOCH YASIN : 8.417 Meter
Shortest DJUMARI - MOCH YASIN : 8.417 Meter

[['Kantor PDAM', 'SUYANTO'], ['ABDULLAH HASAN, H', 'SUKRI'], ['SUKRI', '